# Konfiguration der Google Cloud und Herunterladen der JSON-Datei

In diesem Notebook wird die Google Cloud konfiguriert und eine JSON-Datei heruntergeladen.

In [24]:
# Importieren der benötigten Bibliotheken
from google.cloud import storage
from google.oauth2 import service_account

# Namen des Google Cloud Storage Buckets und der Quelldatei festlegen
bucket_name = 'prod_prototype'
source_blob_name = 'bronze/applicant_data_raw'
local_temp_path = '/tmp/applicant_data_raw.json'

# Dienstkonto-Datei laden
service_account_json = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json'
credentials = service_account.Credentials.from_service_account_file(service_account_json)

# Google Cloud Storage Client initialisieren
client = storage.Client(credentials=credentials, project='prototyp-etl-pipline')
bucket = client.bucket(bucket_name)
blob = bucket.blob(source_blob_name)

# Datei aus dem Bucket herunterladen
blob.download_to_filename(local_temp_path)
print(f"Datei erfolgreich heruntergeladen zu {local_temp_path}.")

# Zuletzt geändertes Datum der heruntergeladenen Datei anzeigen
print(f"Zuletzt geändertes Datum der Datei: {blob.updated}")

Datei erfolgreich heruntergeladen zu /tmp/applicant_data_raw.json.
Zuletzt geändertes Datum der Datei: None


# Initialisiere Spark Session
In diesem Notebook wird die Spark Session vorbereitet.

In [25]:
# Importieren der benötigten Bibliotheken
from pyspark.sql import SparkSession

# Beispiel für das Hinzufügen des GCS Connectors zu einer lokalen Spark-Session
spark = SparkSession.builder \
    .appName("Datatransformation") \
    .config('spark.sql.debug.maxToStringFields', '1000') \
    .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.2") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json") \
    .getOrCreate()
    
# Überprüfen der SparkSession
spark

# Enttferung von Duplikaten
In diesem Notebook werdne alle doppelten Daten entfernt, leere Zellen bleiben leer

In [26]:
# JSON-Datei in ein Spark DataFrame laden
df = spark.read.json('/tmp/applicant_data_raw.json')

# Anzahl der ursprünglichen Datensätze
initial_count = df.count()

# Datenbereinigung: Entfernen von Duplikaten
df_cleaned = df.dropDuplicates()

# Anzahl der bereinigten Datensätze
cleaned_count = df_cleaned.count()

# Anzahl der gelöschten Duplikate
duplicates_removed = initial_count - cleaned_count
print(f"Anzahl der gelöschten Duplikate: {duplicates_removed}")

# Anzeigen der ersten Zeilen des bereinigten DataFrames
print("Die ersten Zeilen des bereinigten DataFrames:")
df_cleaned.show(200)

Anzahl der gelöschten Duplikate: 0
Die ersten Zeilen des bereinigten DataFrames:
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+----------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|    Nachname|Standort|      Status|           StellenID|   Telefonnummer|Veröffentlichungsdatum

# Standartisierung Telefonnummern
In diesem Notebook werden Telefonnummern vereinheitlich

In [27]:
import phonenumbers
from phonenumbers import NumberParseException, PhoneNumberFormat

def standardize_swiss_phone_number(phone_number):
    try:
        # Parsen der Telefonnummer mit der Annahme, dass es eine Schweizer Nummer ist ('CH' ist der Ländercode für die Schweiz)
        parsed_number = phonenumbers.parse(phone_number, "CH")
        
        # Überprüfen, ob die Nummer eine gültige Schweizer Telefonnummer ist
        if phonenumbers.is_valid_number(parsed_number):
            # Formatieren der Nummer im internationalen Format, inklusive der Ländervorwahl
            standardized_number = phonenumbers.format_number(parsed_number, PhoneNumberFormat.INTERNATIONAL)
            return standardized_number
        else:
            return "Ungültige Telefonnummer"
    except NumberParseException:
        return "Fehler beim Parsen der Telefonnummer"

# Beispielaufruf der Funktion
example_number = "079 123 45 67"
standardized_number = standardize_swiss_phone_number(example_number)
print(f"Standardisierte Telefonnummer: {standardized_number}")

df.show(200)

Standardisierte Telefonnummer: +41 79 123 45 67
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+----------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|    Nachname|Standort|      Status|           StellenID|   Telefonnummer|Veröffentlichungsdatum|    Vorname|  id|
+-------------

In [28]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, regexp_replace, length
import re

# Sicherstellen, dass die Spalte 'Telefonnummer' existiert
if 'Telefonnummer' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Telefonnummer' existiert nicht im DataFrame")

# Zählen der Telefonnummern vor der Transformation
initial_phone_count = df_cleaned.filter(length(col("Telefonnummer")) > 0).count()

 #Entfernen von Sonderzeichen aus der Telefonnummer
def clean_phone_number(phone):
    return re.sub(r'\D', '', phone)

# Umwandeln der Telefonnummern in das gewünschte Format: +1-245-345-7426
df_cleaned = df_cleaned.withColumn("Telefonnummer", 
                                   regexp_replace(
                                       col("Telefonnummer"), 
                                       r"(\d{3})(\d{3})(\d{2})(\d{2})", 
                                       r"+41 $1 $2 $3 $4"
                                   ))

# Zählen der Telefonnummern nach der Transformation
transformed_phone_count = df_cleaned.filter(col("Telefonnummer").rlike(r"\+1-\d{3}-\d{3}-\d{2}-\d{2}")).count()

# Anzahl der durchgeführten Transformationen
transformations_done = transformed_phone_count - initial_phone_count

print(f"Anzahl der durchgeführten Transformationen: {transformations_done}")

# Anzeigen der ersten Zeilen des bereinigten DataFrames
print("Die ersten Zeilen des bereinigten DataFrames:")
df_cleaned.show(200)

Anzahl der durchgeführten Transformationen: -2428
Die ersten Zeilen des bereinigten DataFrames:
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+-------------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|    Nachname|Standort|      Status|           StellenID|      Telefonnummer|V

In [29]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, regexp_replace, length

# Sicherstellen, dass die Spalte 'phone' existiert
if 'Telefonnummer' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Telefonnummer' existiert nicht im DataFrame")

# Zählen der Telefonnummern vor der Transformation
initial_phone_count = df_cleaned.filter(length(col("Telefonnummer")) == 10).count()

# Umwandeln der Telefonnummern in das gewünschte Format: +1-245-345-7426
df_cleaned = df_cleaned.withColumn("Telefonnummer", 
                                   regexp_replace(
                                       col("Telefonnummer"), 
                                       r"(\d{1})(\d{3})(\d{3})(\d{4})", 
                                       r"+1-$2-$3-$4"
                                   ))

# Zählen der Telefonnummern nach der Transformation
transformed_phone_count = df_cleaned.filter(col("Telefonnummer").like("+1-%-%-%")).count()

# Anzahl der durchgeführten Transformationen
transformations_done = transformed_phone_count
print(f"Anzahl der durchgeführten Transformationen: {transformations_done}")

# Anzeigen der ersten Zeilen des bereinigten DataFrames
print("Die ersten Zeilen des bereinigten DataFrames:")
df_cleaned.show(200)

Anzahl der durchgeführten Transformationen: 0
Die ersten Zeilen des bereinigten DataFrames:
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+-------------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|    Nachname|Standort|      Status|           StellenID|      Telefonnummer|Veröf

# Auffüllen leerer Adresszellen

In diesem Notebook werden leere Zellen in der Spalte Adresse mit dem Platzhalter "Unknown" gefüllt.


In [30]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, when

# Sicherstellen, dass die Spalte 'Adresse' existiert
if 'Adresse' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Adresse' existiert nicht im DataFrame")

# Zählen der leeren Adresszellen vor der Transformation
initial_empty_address_count = df_cleaned.filter(col("Adresse").isNull()).count()

# Auffüllen leerer Zellen in der Spalte 'Adresse' mit 'Unknown'
df_cleaned = df_cleaned.withColumn('Adresse', when(col('Adresse').isNull(), 'Unknown').otherwise(col('Adresse')))

# Zählen der leeren Adresszellen nach der Transformation
final_empty_address_count = df_cleaned.filter(col("Adresse") == 'Unknown').count()

# Anzahl der durchgeführten Auffüllungen
fillings_done = final_empty_address_count
print(f"Anzahl der aufgefüllten Adresszellen: {fillings_done}")

# Anzeigen der ersten Zeilen des bereinigten DataFrames
print("Die ersten Zeilen des bereinigten DataFrames:")
df_cleaned.show(200)

Anzahl der aufgefüllten Adresszellen: 127
Die ersten Zeilen des bereinigten DataFrames:
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+-------------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|    Nachname|Standort|      Status|           StellenID|      Telefonnummer|Veröffent

# Auffüllen leerer Datumszellen

In diesem Notebook werden leere Zellen in den Spalten Geburtsdatum, Bewerbungsdatum, Veröffentlichungsdatum und Einstellungsdatum mit dem Platzhalter "1900-01-01" gefüllt.

In [31]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, when

# Sicherstellen, dass die Spalten 'Geburtsdatum', 'Bewerbungsdatum', 'Veröffentlichungsdatum' und 'Einstellungsdatum' existieren
if 'Geburtsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Geburtsdatum' existiert nicht im DataFrame")
if 'Bewerbungsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Bewerbungsdatum' existiert nicht im DataFrame")
if 'Veröffentlichungsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Veröffentlichungsdatum' existiert nicht im DataFrame")
if 'Einstellungsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Einstellungsdatum' existiert nicht im DataFrame")

# Zählen der leeren Datumszellen vor der Transformation
initial_empty_birthdate_count = df_cleaned.filter(col("Geburtsdatum").isNull()).count()
initial_empty_applicationdate_count = df_cleaned.filter(col("Bewerbungsdatum").isNull()).count()
initial_empty_publishdate_count = df_cleaned.filter(col("Veröffentlichungsdatum").isNull()).count()
initial_empty_hiredate_count = df_cleaned.filter(col("Einstellungsdatum").isNull()).count()

# Auffüllen leerer Zellen in den Spalten mit '1900-01-01'
df_cleaned = df_cleaned.withColumn('Geburtsdatum', when(col('Geburtsdatum').isNull(), '1900-01-01').otherwise(col('Geburtsdatum')))
df_cleaned = df_cleaned.withColumn('Bewerbungsdatum', when(col('Bewerbungsdatum').isNull(), '1900-01-01').otherwise(col('Bewerbungsdatum')))
df_cleaned = df_cleaned.withColumn('Veröffentlichungsdatum', when(col('Veröffentlichungsdatum').isNull(), '1900-01-01').otherwise(col('Veröffentlichungsdatum')))
df_cleaned = df_cleaned.withColumn('Einstellungsdatum', when(col('Einstellungsdatum').isNull(), '1900-01-01').otherwise(col('Einstellungsdatum')))

# Zählen der leeren Datumszellen nach der Transformation
final_empty_birthdate_count = df_cleaned.filter(col("Geburtsdatum") == '1900-01-01').count()
final_empty_applicationdate_count = df_cleaned.filter(col("Bewerbungsdatum") == '1900-01-01').count()
final_empty_publishdate_count = df_cleaned.filter(col("Veröffentlichungsdatum") == '1900-01-01').count()
final_empty_hiredate_count = df_cleaned.filter(col("Einstellungsdatum") == '1900-01-01').count()

# Anzahl der durchgeführten Auffüllungen
birthdate_fillings_done = final_empty_birthdate_count
applicationdate_fillings_done = final_empty_applicationdate_count
publishdate_fillings_done = final_empty_publishdate_count
hiredate_fillings_done = final_empty_hiredate_count

print(f"Anzahl der aufgefüllten Geburtsdatum Zellen: {birthdate_fillings_done}")
print(f"Anzahl der aufgefüllten Bewerbungsdatum Zellen: {applicationdate_fillings_done}")
print(f"Anzahl der aufgefüllten Veröffentlichungsdatum Zellen: {publishdate_fillings_done}")
print(f"Anzahl der aufgefüllten Einstellungsdatum Zellen: {hiredate_fillings_done}")

# Anzeigen der ersten Zeilen des bereinigten DataFrames
print("Die ersten Zeilen des bereinigten DataFrames:")
df_cleaned.show(200)

Anzahl der aufgefüllten Geburtsdatum Zellen: 124
Anzahl der aufgefüllten Bewerbungsdatum Zellen: 0
Anzahl der aufgefüllten Veröffentlichungsdatum Zellen: 123
Anzahl der aufgefüllten Einstellungsdatum Zellen: 121
Die ersten Zeilen des bereinigten DataFrames:
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+-------------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma

# Auffüllen leerer Bewertungszellen
In diesem Notebook werden leere Zellen in den Spalten Bewertung Assessment, Bewertung Erstes Gespräch und Bewertung Prescreening mit dem Platzhalter Unknown gefüllt.

In [32]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, when

# Sicherstellen, dass die Spalten existieren
if 'Bewertung Assessment' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Bewertung Assessment' existiert nicht im DataFrame")
if 'Bewertung Erstes Gespräch' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Bewertung Erstes Gespräch' existiert nicht im DataFrame")
if 'Bewertung Prescreening' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Bewertung Prescreening' existiert nicht im DataFrame")

# Zählen der leeren Bewertungszellen vor der Transformation
initial_empty_assessment_count = df_cleaned.filter(col("Bewertung Assessment").isNull()).count()
initial_empty_first_interview_count = df_cleaned.filter(col("Bewertung Erstes Gespräch").isNull()).count()
initial_empty_prescreening_count = df_cleaned.filter(col("Bewertung Prescreening").isNull()).count()

# Auffüllen leerer Zellen in den Spalten mit 'Unknown'
df_cleaned = df_cleaned.withColumn('Bewertung Assessment', when(col('Bewertung Assessment').isNull(), 'Unknown').otherwise(col('Bewertung Assessment')))
df_cleaned = df_cleaned.withColumn('Bewertung Erstes Gespräch', when(col('Bewertung Erstes Gespräch').isNull(), 'Unknown').otherwise(col('Bewertung Erstes Gespräch')))
df_cleaned = df_cleaned.withColumn('Bewertung Prescreening', when(col('Bewertung Prescreening').isNull(), 'Unknown').otherwise(col('Bewertung Prescreening')))

# Zählen der 'Unknown'-Bewertungszellen nach der Transformation
final_unknown_assessment_count = df_cleaned.filter(col("Bewertung Assessment") == 'Unknown').count()
final_unknown_first_interview_count = df_cleaned.filter(col("Bewertung Erstes Gespräch") == 'Unknown').count()
final_unknown_prescreening_count = df_cleaned.filter(col("Bewertung Prescreening") == 'Unknown').count()

# Anzahl der durchgeführten Auffüllungen
assessment_fillings_done = final_unknown_assessment_count - (initial_empty_assessment_count - final_unknown_assessment_count)
first_interview_fillings_done = final_unknown_first_interview_count - (initial_empty_first_interview_count - final_unknown_first_interview_count)
prescreening_fillings_done = final_unknown_prescreening_count - (initial_empty_prescreening_count - final_unknown_prescreening_count)

print(f"Anzahl der aufgefüllten Bewertung Assessment Zellen: {assessment_fillings_done}")
print(f"Anzahl der aufgefüllten Bewertung Erstes Gespräch Zellen: {first_interview_fillings_done}")
print(f"Anzahl der aufgefüllten Bewertung Prescreening Zellen: {prescreening_fillings_done}")

# Anzeigen der ersten Zeilen des bereinigten DataFrames
print("Die ersten Zeilen des bereinigten DataFrames:")
df_cleaned.show(200)

Anzahl der aufgefüllten Bewertung Assessment Zellen: 113
Anzahl der aufgefüllten Bewertung Erstes Gespräch Zellen: 119
Anzahl der aufgefüllten Bewertung Prescreening Zellen: 122
Die ersten Zeilen des bereinigten DataFrames:
+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+-------------------+----------------------+-----------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Int

# Hochladen der bereinigten Daten in den Google Cloud Storage

In diesem Notebook werden die bereinigten Daten wieder in den Google Cloud Storage hochgeladen.


In [37]:
# Importieren der benötigten Bibliotheken
import os

# Funktion zum Hochladen eines Verzeichnisses zu Google Cloud Storage
def upload_directory_to_gcs(bucket_name, source_directory, destination_blob_prefix):
    client = storage.Client(credentials=credentials, project='prototyp-etl-pipline')
    bucket = client.bucket(bucket_name)
    
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, source_directory)
            blob_path = os.path.join(destination_blob_prefix, relative_path)
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)
            print(f"File {local_path} uploaded to {blob_path}.")

# Namen des Google Cloud Storage Buckets und der bereinigten Datei festlegen
bucket_name = 'prod_prototype'
cleaned_blob_prefix = 'silver/sensitive/'
cleaned_temp_path = '/tmp/applicant_data_cleaned.parquet'

# Dienstkonto-Datei laden
service_account_json = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json'
credentials = service_account.Credentials.from_service_account_file(service_account_json)

# DataFrame in eine Parquet-Datei speichern
parquet_output_path = '/tmp/applicant_data_cleaned.parquet'
df_cleaned.write.mode('overwrite').parquet(parquet_output_path)
print(f"Bereinigte Daten erfolgreich in Parquet gespeichert zu {parquet_output_path}.")


# Hochladen des Parquet-Verzeichnisses
upload_directory_to_gcs(bucket_name, cleaned_temp_path, cleaned_blob_prefix)
print(f"Bereinigte Dateien erfolgreich hochgeladen zu {cleaned_blob_prefix} in bucket {bucket_name}.")

# Lokaler Pfad zur CSV-Datei
csv_output_path = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Output/applicant_data_final.csv'

# DataFrame in eine CSV-Datei speichern
df_cleaned.write.mode('overwrite').option("header", "true").csv(csv_output_path)
print(f"Bereinigte Daten erfolgreich in CSV gespeichert zu {csv_output_path}.")


Bereinigte Daten erfolgreich in Parquet gespeichert zu /tmp/applicant_data_cleaned.parquet.
File /tmp/applicant_data_cleaned.parquet/._SUCCESS.crc uploaded to silver/sensitive/._SUCCESS.crc.
File /tmp/applicant_data_cleaned.parquet/part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet uploaded to silver/sensitive/part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet.
File /tmp/applicant_data_cleaned.parquet/_SUCCESS uploaded to silver/sensitive/_SUCCESS.
File /tmp/applicant_data_cleaned.parquet/.part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet.crc uploaded to silver/sensitive/.part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet.crc.
Bereinigte Dateien erfolgreich hochgeladen zu silver/sensitive/ in bucket prod_prototype.
Bereinigte Daten erfolgreich in CSV gespeichert zu /Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Output/applicant_data_final.csv.
